In [1]:
sc

In [2]:
rdd = sc.textFile("/home/yihao/Downloads/spark-2.1.0-bin-hadoop2.6/README.md")

In [8]:
rdd.take(10)

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a',
 u'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 u'MLlib for machine learning, GraphX for graph processing,',
 u'and Spark Streaming for stream processing.',
 u'',
 u'<http://spark.apache.org/>']

In [9]:
rdd.map(lambda x: x.lower()).flatMap(lambda x: x.split()).take(10)

[u'#',
 u'apache',
 u'spark',
 u'spark',
 u'is',
 u'a',
 u'fast',
 u'and',
 u'general',
 u'cluster']

In [10]:
rdd.flatMap(lambda x: x.split()) \
    .map(lambda x: (x.lower(),1)) \
    .reduceByKey(lambda x,y: x+y) \
    .take(10)

[(u'when', 1),
 (u'alternatively,', 1),
 (u'"local"', 1),
 (u'including', 4),
 (u'computation', 1),
 (u'note', 1),
 (u'submit', 1),
 (u'using:', 1),
 (u'guidance', 2),
 (u'environment', 1)]

In [11]:
rdd.flatMap(lambda x: x.split()) \
    .map(lambda x: (x.lower(),1)) \
    .groupByKey() \
    .mapValues(lambda x:sum(x)) \
    .take(10)

[(u'when', 1),
 (u'alternatively,', 1),
 (u'"local"', 1),
 (u'including', 4),
 (u'computation', 1),
 (u'note', 1),
 (u'submit', 1),
 (u'using:', 1),
 (u'guidance', 2),
 (u'environment', 1)]

In [12]:
chunks = sc.textFile("/home/yihao/Downloads/spark-2.1.0-bin-hadoop2.6/README.md",minPartitions=3)
chunks.getNumPartitions()

3

In [13]:
def mapper(klvls):
    for k1v1 in klvls:
        for word in k1v1.split():
            yield (word.lower(),1)
            
wc = chunks.mapPartitions(mapper).reduceByKey(lambda x,y:x+y)
wc.take(2)

[(u'when', 1), (u'alternatively,', 1)]

In [14]:
import heapq
def mapper2(counts):
    yield (0,heapq.nlargest(3,counts,key=lambda x:x[1]))

def reducer2(top31,top32):
    return heapq.nlargest(3,top31 + top32, key=lambda x:x[1])

top3 = wc.mapPartitions(mapper2).reduceByKey(reducer2)
top3.collect()

[(0, [(u'the', 25), (u'to', 19), (u'spark', 16)])]

In [15]:
import heapq
def mapper2(counts):
    yield heapq.nlargest(3,counts,key=lambda x:x[1])
    
def reducer2(top31,top32):
    return heapq.nlargest(3,top31+top32,key=lambda x:x[1])

top3 = wc.mapPartitions(mapper2).reduce(reducer2)
top3

[(u'the', 25), (u'to', 19), (u'spark', 16)]

In [18]:
trips = sc.textFile('citibike.csv').cache()

In [19]:
def mapper3(partId,records):
    if partId==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for trip in reader:
        yield trip[2]
        

output= trips.mapPartitionsWithIndex(mapper3)
output.take(4)

['801', '379', '2474', '818']